# OpenSearch Hybrid 검색을 통한 RAG
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 이 노트북은 이전 노트북인 "02_1_KR_RAG_OpenSearch_Keyword.ipynb" 이 완료 되었다고 가정 합니다.
    - 오픈서치 인텍스 관련 정보를 참조 합니다.

### Methods and Resources for Hybrid search with Re-ranking 
- Score Normalization
    - [MinMax based](https://towardsdatascience.com/text-search-vs-vector-search-better-together-3bd48eb6132a)
    - [z-socre based](https://towardsdatascience.com/hybrid-search-2-0-the-pursuit-of-better-search-ce44d6f20c08)
- Reciprocal Rank Fusion (RRF)
    - [Paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)
    - [Description](https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d)
- [LangChain API for Ensemble Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다


In [1]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [2]:
opensearch_user_id = 'raguser'
opensearch_user_password = 'MarsEarth1!'
domain_name = "rag-hol-871cbabb"
opensearch_domain_endpoint = "https://search-rag-hol-871cbabb-u3mzw575hng5fi6i3ys2q2253a.us-east-1.es.amazonaws.com"

In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [4]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2.1)

In [5]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7fdeebe4c490>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7fdf1580df30>])

## Embedding 모델 선택

In [7]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [8]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


# 3. LangChain OpenSearch VectorStore 생성 
## 선수 조건
- 이전 노트북 02_1_KR_RAG_OpenSearch_Keyword.ipynb 또는 02_1_KR_RAG_OpenSearch_Semantic.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.

## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 02_1_rag_opensearch_lexical_kr.ipynb를 통해 opensearch의 "genai-demo-index-v1-with-tokenizer" 인덱스가 생성되어 있어야 합니다.

In [9]:
http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password
index_name = "genai-demo-index-v1-with-tokenizer"

## LangChain OpenSearch VectorStore 생성

In [10]:
from langchain.vectorstores import OpenSearchVectorSearch
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Client 생성

In [11]:
from utils.opensearch import opensearch_utils
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 형태소 분석기 (nori_tokenizer) 사용하기 in Opensearch
- 형태소 분석기에 대한 자세한 사항은 02_1_KR_RAG_OpenSearch_Keyword.ipynb 참고

### 인덱스 확인 (tokenization 확인)

In [12]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)

{'genai-demo-index-v1-with-tokenizer': {'aliases': {},
                                        'mappings': {'properties': {'metadata': {'properties': {'row': {'type': 'long'},
                                                                                                'source': {'type': 'keyword'},
                                                                                                'timestamp': {'type': 'float'},
                                                                                                'type': {'type': 'keyword'}}},
                                                                    'text': {'analyzer': 'my_analyzer',
                                                                             'fields': {'keyword': {'ignore_above': 256,
                                                                                                    'type': 'keyword'}},
                                                                             'type': 'text'},
  

# 4. 오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [13]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter

## (1) OpenSearch Vector 검색

### 프로프트 템플릿 생성

In [14]:
from utils.rag import run_RetrievalQA, show_context_used
from langchain.prompts import PromptTemplate

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [15]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """


Human: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


### 필터 생성

In [16]:
filter01 = "인터넷뱅킹"
filter02 = "신한은행"
query = "중지된 경우 이체"

boolean_filter = [
    {"term": {"metadata.type": filter01}},
    {"term": {"metadata.source": filter02}},
]

pprint(boolean_filter)

[{'term': {'metadata.type': '인터넷뱅킹'}}, {'term': {'metadata.source': '신한은행'}}]


### LangChain RetrievalQA 를 통해 실행
- 아래와 같이 top_k = 5 를 실행하면 해당 관련 Context 를 가져오지 못하고 , "주어진 내용에서 찾을 수 없습니다" 라고 답변 함.

In [17]:
result = run_RetrievalQA(
    query=query,
    boolean_filter=boolean_filter,
    llm=llm_text,
    prompt=PROMPT,
    vector_db=vector_db,
    verbose=True,
    k=1
)

print("##################################")
print("query: ", query)
print("boolean_filter: ", boolean_filter)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(result['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(result['source_documents'])



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the context, inside <context></context> XML tags.

<context>
ask: 이체한도란 무엇인가요?
Information: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는 이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이 1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③ 최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
중지된 경우 이체

If the answer is not in the

## (2) OpenSearch Keyword 검색
- "minimum_should_match=50" 세팅을 하고, 관련 컨텍스트를 찾아 오지 못합니다.

In [18]:
from utils.rag import retriever_utils

In [19]:
from utils.opensearch import opensearch_utils
from langchain.chains.question_answering import load_qa_chain

In [20]:
filter01 = "인터넷뱅킹"
filter02 = "신한은행"
query = "중지된 경우 이체"

search_keyword_result = retriever_utils.get_lexical_similar_docs(
    query=query,
    minimum_should_match=0,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    index_name=index_name,
    os_client=os_client,
    k=1,
    hybrid=False
)
print(search_keyword_result)

[Document(page_content='ask: 예약이체를 했는데 이체일에 송금이 안된 경우는?\nInformation: 모든 예약이체 등록건은 예약이체 지정일 지정된 시간(오전 7시, 오전 9시, 오전 11시, 오후 1시, 오후 3시, 오후 5시, 오후 7시, 오후 9시) 이전에 출금할 계좌에 송금액이 입금이 되어야 이체가 실행될 수 있습니다.\r\n이체가 되지 않았다고 판단되실 경우에는 반드시 먼저 "예약이체처리결과조회" 또는 "계좌 거래내역"을 확인 하시어 이체여부 및 출금여부를 확인하시기 바랍니다. 또한 예약이체 신청시 SMS 결과통보신청을 하셨다면 요청하신 휴대폰번호로 수신된 문자를 확인해주시기 바라며, 정해진 시간에 이체되지 않은 경우 신한은행 고객상담센터 ☎ 1599-8000번으로 문의 주시기 바랍니다.', metadata={'source': '신한은행', 'row': 37, 'type': '인터넷뱅킹', 'timestamp': 1702378552.563887, 'id': 'e3d5a802-1424-43fc-89e1-f4e3875b8ffb'})]


In [21]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

answer = chain.run(
    input_documents=search_keyword_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:



Human: Here is the context, inside <context></context> XML tags.

<context>
ask: 예약이체를 했는데 이체일에 송금이 안된 경우는?
Information: 모든 예약이체 등록건은 예약이체 지정일 지정된 시간(오전 7시, 오전 9시, 오전 11시, 오후 1시, 오후 3시, 오후 5시, 오후 7시, 오후 9시) 이전에 출금할 계좌에 송금액이 입금이 되어야 이체가 실행될 수 있습니다.
이체가 되지 않았다고 판단되실 경우에는 반드시 먼저 "예약이체처리결과조회" 또는 "계좌 거래내역"을 확인 하시어 이체여부 및 출금여부를 확인하시기 바랍니다. 또한 예약이체 신청시 SMS 결과통보신청을 하셨다면 요청하신 휴대폰번호로 수신된 문자를 확인해주시기 바라며, 정해진 시간에 이체되지 않은 경우 신한은행 고객상담센터 ☎ 1599-8000번으로 문의 주시기 바랍니다.
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
중지된 경우 이체

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:


### 키워드 검색 결과 (search_keyword_result)
bm25 score는 max_value로 normalization 되어 있음 (score range 0 - 1)

In [22]:
search_keyword_result

[Document(page_content='ask: 예약이체를 했는데 이체일에 송금이 안된 경우는?\nInformation: 모든 예약이체 등록건은 예약이체 지정일 지정된 시간(오전 7시, 오전 9시, 오전 11시, 오후 1시, 오후 3시, 오후 5시, 오후 7시, 오후 9시) 이전에 출금할 계좌에 송금액이 입금이 되어야 이체가 실행될 수 있습니다.\r\n이체가 되지 않았다고 판단되실 경우에는 반드시 먼저 "예약이체처리결과조회" 또는 "계좌 거래내역"을 확인 하시어 이체여부 및 출금여부를 확인하시기 바랍니다. 또한 예약이체 신청시 SMS 결과통보신청을 하셨다면 요청하신 휴대폰번호로 수신된 문자를 확인해주시기 바라며, 정해진 시간에 이체되지 않은 경우 신한은행 고객상담센터 ☎ 1599-8000번으로 문의 주시기 바랍니다.', metadata={'source': '신한은행', 'row': 37, 'type': '인터넷뱅킹', 'timestamp': 1702378552.563887, 'id': 'e3d5a802-1424-43fc-89e1-f4e3875b8ffb'})]

## (3) OpenSearch Hybrid 검색
- 아래는 결론적으로 query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?" 에 답변을 제공 합니다.
    - 시멘티 서치에서 LangChain 의 RetrievalQA 사용 대신에 , LangChain OpenSearch Vector Store의 similarity_search_with_score 를 사용 하여 제일 첫번재에 해당 답변이 검색이 됩니다.
    - 키워드 검색에서는 해당 qeury 가 검색이 되지 않습니다.
    - 이후에 시멘틱 검색, 키워드 검색에 가중치를 [0.3 , 0.5] 를 주어서 Ensembe 을 하여 랭킹을 하면, top_k = 5 에서 5번재로 Context 로 포함이 되어서 , 최종 검색이 됩니다.

#### OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) 위의 두 개의 결과에 랭킹 알고리즘 ("RRF" 혹은 "simple_weighted"] 을 통해, 스코어를 계산하여 정렬하여 제공 함.
    - 여기서는 simple_weighted 사용 함.

스코어 표준화 예시

<pre>
Docs    Score	    Normalized-Score

Doc1: 	0.0083		1.0

Doc2: 	0.0074		0.8900

Doc3: 	0.0071		0.8585

Example: 0.0074 / 0.0083 = 0.8900
</pre>

In [23]:
from functools import partial
from multiprocessing.pool import ThreadPool
#from utils.rag import get_semantic_similar_docs, get_lexical_similar_docs, get_ensemble_results, get_rerank_docs

In [24]:
%%time

filter01 = "인터넷뱅킹"
filter02 = "신한은행"
query = "중지된 경우 이체"

search_hybrid_result = retriever_utils.search_hybrid(
    query=query,
    llm_emb=llm_emb,
    vector_db=vector_db,
    k=5,
    index_name=index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[.5, .5], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=True
)

answer = chain.run(
    input_documents=search_hybrid_result,
    question=query
)

##############################
async_mode
##############################
True
##############################
reranker
##############################
False
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
similar_docs_semantic
##############################
[(Document(page_content='ask: 이체한도란 무엇인가요?\nInformation: 이체한도란 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 통하여 이체할 수 있는 거래한도를 말합니다. ① 1일/1회 이체한도 1일 이체한도란 하루에 거래할 수 있는 이체금액의 합을 말하여, 1회 이체한도란 1회에 이체하실 수 있는 금액의 한도를 말합니다. ② 통합이체한도 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등을 고객님이 사용하시는 경우 고객님이 1일 또는 1회에 이체하실 수 있는 이체한도를 말합니다. 인터넷뱅킹, 폰뱅킹, 모바일뱅킹 등에서 고객님이 거래하신 이체금액의 합이 통합이체한도의 범위를 초과할 수 없습니다. ③ 최고이체한도 금융감독원에서는 개인의 전자금융 시 이용할 수 있는 이체한도의 가이드라인을 정하고 있습니다.(인터넷뱅킹 : 1일 5억원, 1회 1억원 이내)', metadata={'source': '신한은행', 'row': 39, 'type': '인터넷뱅킹', 'timestamp': 1702378552.5638924, 'id':

In [25]:
print(f'question: {query}')
print(f'response: {answer}')

question: 중지된 경우 이체
response:  주어진 내용에서, 

12개월 장기미이체로 이체서비스가 중지된 경우 어떻게 해제하나요?

라는 질문에 대한 답변이 있습니다.

최근 12개월동안 이체서비스를 사용하지 않은 경우 이체서비스가 중단되며, 이체를 원하시는 경우 인터넷뱅킹, 폰뱅킹, 신한쏠(SOL)에서 이체정지 해제를 하실 수 있습니다. 

1) ARS 추가인증 가능한 경우 - 인터넷뱅킹이나 쏠(SOL)에서 '장기미사용 정지해제' 메뉴를 통해 해제
2) ARS 추가인증 불가한 경우 - 쏠(SOL)에서 비대면 실명인증 후 해제

이와 같이 중지된 이체서비스를 해제할 수 있습니다.
